In [5]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# Table of contents:
* [What is Logistic Regression](#1)
* [Overview of the algorithm](#2)
* [A word on the Logistic function](#3)
* [Logistic Regression is effective when](#4)
* [When to watch out for?](#5)
* [Code](#6)
* [Metric used to evaluate](#7)
* [Interview questions](#8)


# What is Logistic regression? <a class="anchor" id="1"></a>

Logistic regression is a statistical modeling technique used to analyze the relationship between a dependent variable and one or more independent variables, where the dependent variable represents a binary outcome or a categorical outcome with multiple classes. 

# Overview of the algorithm <a class="anchor" id="2"></a>

* *Data processing:* <br>
    Since the logistic regression is sensitive to the scales of the features of the input data, we need to normalize the data. If we don't normalize the data, some input variables with higher scales will end up having uneven influence on the model.
* *Removing outliers* <br>
    Unlike decision trees, logistic regression is sensitive to outliers. Thus, it is important to visualize the data and remove the outliers.
* *Linear combination of independent variables:* <br>
    Logistic regression creates a linear combination of input variables by multiplying them to the coefficients of the weights. The results are then summed up.
* *Sigmoid function:* <br>
    The linear combination output is then non-linearly mapped to a value between $[0,1]$ representing the estimated probability of the outcome.
* *Gradient Descent or Maximum Likelihood Estimation:* <br>
    The coefficients for the independent variables which we called weights are estimated using **Gradient Descent** or **Maximum likelihood estimation**. We want to find the parameters which maximize the probability of occurence of the data given its predictions.
* *Evaluation:* <br>
    The model can be evaluaed using several metrics such as acuracy or precision & recall.

# A word on the Logistic function <a class="anchor" id="3"></a>

**Logistic function:** 

$$ f(x)= {L \over (1+e^{-k(x−x0)}) }$$ 

In a logistic fundtion : <br>
 * $L$ controls the max value the function can take
 * $k$ controls the steepness of the curve
 * $x_0$ controls where the function is centered on the x axis

**Sigmoid function:**

$$ S(x) = { 1 \over (1 + e^{-x}) }$$

Sigmoid function is a special case of logistic function where, 
 * max value the function can take is $1$
 * $x_0 = 0$ i.e. the function is centered at 0
 * The steepnes of the curve is $1$

The Sigmoid function takes an unbounded real value and maps it to a real value between 0 and 1. <br>

**But what makes it a good choice for modeling probabilities? <br>**
 * We can map the linear combination of input variables to a value to $[0, 1]$.
 * It is a continous function.
 * It is non linear
 * It is monotonously increasing.
 * It is a differentiable function.
 
**What are some limitations of the sigmoid function?**
 * It gives rise to vanishing gradients: when the x values are large in magnitude (towards the extremeties), it tends to produce a very small change in the Y value. Hence, the gradients produced are near 0.


# Code: Binary Logistic Regression


 

## Data preparation

In [2]:
from sklearn.datasets import load_breast_cancer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# load breast cancer dataset
data = load_breast_cancer()

# split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, test_size=0.2, random_state=42)


ModuleNotFoundError: No module named 'sklearn'

* Note that in this code, we use **Xavier initialization** for initialize our weights. We can also use normal initialization for this. You can run it with either and see the effect on the process. Random initialization converges slower and yields less accuracy as compared to Xavier initialization. <br>
 Checkout the function: `xavier_init`
 * We use gradient descent here to optimize: <br>
   Let $X$ be our input data consistting of $N$ samples, $W$ be our weights and $b$ be the bias, and $Y_T$ be the ground truth. <br>
   $$ H = W.X + b $$ <br>
   $$ Y_{pred} = Sigmoid(H) $$  <br>
   $$ BCE_{loss} = L = -{\sum_{i=1}^N( Y_T{log}(Y_pred) + (1 - Y_T) log (1 - Y_pred) ) \over N }$$ <br>
   So, now when we compute the gradient wrt loss, we get, 
   $$ {{\partial L} \over {\partial Y_{pred}}}  = { {-1\over N}[ {Y_T \over Y_pred} - {(1-Y_T) \over (1-Y_pred)} ] }$$
   Partial gradient of predictions over sigmoid output<br>
   (check stack overflow or derive yourself, its easy takes a few equation manipulations)
   $$ {\partial Y_{pred} \over \partial H}  = H * (1 - H)$$ <br>
   Partial gradient of $H$ over weights $W$
   $$ {\partial H \over \partial W} = X $$
   Partial gradient of $H$ over bias $b$
   $$ {\partial H \over \partial b} = 1 $$
   
   Now, using **chain-rule** of derivatives, we can stitch to find the gradient of loss $L$ wrt weights $W$ and bias $W$,
   
   $$ {\partial H \over \partial W} = {\partial L \over \partial Y_{pred}} {\partial Y_{pred} \over \partial H} {\partial H \over \partial W} $$
   
   Similary for bias $b$, 

   $$ {\partial H \over \partial b} = {\partial L \over \partial Y_{pred}} {\partial Y_{pred} \over \partial H} {\partial H \over \partial b} $$
   
 Checkout the functions `_forward_pass` and `_update_weights` on how these equations are used

In [55]:
class BinaryLogisticRegression:
    
    def __init__(self, lr=1e-4, niter=5000, lambda_reg=0.0):
        self.lr = lr
        self.niter = niter
        self.lambda_reg = lambda_reg
        self.num_features = None
        self.weights = None
        self.bias = 0
        self.eps = 1e-6
        self.loss_history = None
        
    def fit(self, X, Y, init_wt='random'):
        self.nsamples, self.nfeatures = X.shape
        if init_wt == 'random':
            self.weights = np.random.rand(self.nfeatures)
        elif init_wt == 'xavier':
            self.weights = self.xavier_init(X.shape, (self.nfeatures, 1))
        else:
            raise ValueError("Invalid initialization given for weights")

        self.weights = self.weights.flatten()
        self.bias = 0
        self.loss_history = []
        X = self._normalize_data(X)
        for i in range(self.niter):
            ypred = self._forward_pass(X)
            loss = self._bceloss(ypred, Y)
            self.loss_history.append(loss)
            if i % 500 == 0:   
                print(f"iter: {i+1} --> Loss: {loss}")
            self._update_weights(X, Y, ypred)
       
    
    def predict(self, Xtest, Ytest):
        Xtest = self._normalize_data(Xtest)
        Ypred = self._forward_pass(Xtest)
        th = 0.5
        prP = Ypred > th 
        prN = Ypred <= th
        gtP = Ytest == 1
        gtN = Ytest == 0
        tp = np.sum(prP & gtP)
        tn = np.sum(prN & gtN)
        fp = np.sum(prP & gtN)
        fn = np.sum(prN & gtP)
        precision = tp/(tp + fp)
        recall = tp/(tp + fn)
        accuracy = tp/len(Xtest)
        print(f"Precision = {precision}")
        print(f"recall = {recall}")
        return Ypred
    
    def xavier_init(self, input_shape, output_shape):
        """
        Xavier initialization for weights in a linear layer
        :param input_shape: tuple, shape of input to layer
        :param output_shape: tuple, shape of output from layer
        :return: ndarray, initialized weights for layer
        """
        scale = np.sqrt(2 / (input_shape[0] + output_shape[0]))
        return np.random.randn(input_shape[1], output_shape[1]) * scale
    
    
    def _normalize_data(self, X):
        """
        Normalizes the data by subtracing the mean and dividing by std.
        Note that this doesn't fix the range between -1 to 1. 
        X: data with shape: (num_samples, num_feats)
        """
        mu = np.mean(X, axis=0, keepdims=True)
        std = np.std(X, axis=0, keepdims=True)
        Xnew = (X - mu) / (std + self.eps)
        return Xnew
    
    
    
    
    def _bceloss(self, ypred, ytrue):
        """
        BInary cross entropy loss
        """
        assert len(ypred) == len(ytrue) and len(ypred) > 0
        nsamples = len(ypred)
        loss = (-1./nsamples) * np.sum( (ytrue)* np.log(ypred + self.eps) + (1. - ytrue) * np.log(1 - ypred + self.eps) )
        reg = (self.lambda_reg/2) * np.sum(self.weights ** 2)
        return loss + reg
                                 
    
    def _forward_pass(self, X):
        H = np.dot(X, self.weights) + self.bias
        # Clip predicted probabilities to prevent overflow
        ypred = np.clip(self._sigmoid(H), self.eps, 1 - self.eps)
        return ypred
        
    
    def _update_weights(self, X, Yt, Yp):
        # dLoss/dy = -1/N[ Yt/Yp - (1-Yt)/(1-Yp) ]
        # dY/dH = H*(1-H)
        # dH/dW = X
        # dH/dB = 1
        nsamples = len(X)
        dLoss_dy = (-1./nsamples) * np.sum( Yt/(Yp + self.eps) - (1 - Yt)/(1 - Yp + self.eps) )
        dy_dH = Yp * (1 - Yp)
        dH_dW = X
        dW = np.dot(dLoss_dy * dy_dH, dH_dW) + self.lambda_reg * self.weights # mine
        dB = np.mean(dLoss_dy * dy_dH * 1.)
        self.weights = self.weights - self.lr * dW
        self.bias = self.bias - self.lr * dB
        
    
    def _sigmoid(self, X):
        X = np.clip(X, -500, 500) # we clip the input to 
        return 1. / (1. + np.exp(-X))
     
        

In [56]:
## Let's train the model

In [57]:
blr = BinaryLogisticRegression(lr=1e-3, niter=10000, lambda_reg=0.5)
blr.fit(X_train, Y_train)


iter: 1 --> Loss: 0.053960030612626136
iter: 501 --> Loss: 2275.1495605406435
iter: 1001 --> Loss: 1378.1400801191146
iter: 1501 --> Loss: 834.1443545827769
iter: 2001 --> Loss: 505.895954097304
iter: 2501 --> Loss: 320.00074065801334
iter: 3001 --> Loss: 231.90421043888463
iter: 3501 --> Loss: 189.68302851950628
iter: 4001 --> Loss: 166.26852639179222
iter: 4501 --> Loss: 153.2239452213424
iter: 5001 --> Loss: 143.8548887025751
iter: 5501 --> Loss: 135.53265853514893
iter: 6001 --> Loss: 127.76577939009245
iter: 6501 --> Loss: 120.43877531490958
iter: 7001 --> Loss: 113.50396349967983
iter: 7501 --> Loss: 106.93316949102513
iter: 8001 --> Loss: 100.70532935650601
iter: 8501 --> Loss: 94.80237782496042
iter: 9001 --> Loss: 89.20780302861064
iter: 9501 --> Loss: 83.90976880615658


In [58]:
Ypred = blr.predict(X_test, Y_test)

Precision = 1.0
recall = 1.0


In [62]:
np.stack([Ypred, Y_test], axis=1)

array([[9.99932852e-01, 1.00000000e+00],
       [1.00000000e-06, 0.00000000e+00],
       [9.99951860e-01, 1.00000000e+00],
       [9.99999000e-01, 1.00000000e+00],
       [1.00000000e-06, 0.00000000e+00],
       [9.91267379e-01, 1.00000000e+00],
       [9.99999000e-01, 2.00000000e+00],
       [9.99999000e-01, 2.00000000e+00],
       [1.00000000e-06, 0.00000000e+00],
       [9.99529572e-01, 1.00000000e+00],
       [9.99999000e-01, 2.00000000e+00],
       [9.99999000e-01, 2.00000000e+00],
       [1.00000000e-06, 0.00000000e+00],
       [9.99999000e-01, 2.00000000e+00],
       [1.00000000e-06, 0.00000000e+00],
       [9.99980219e-01, 1.00000000e+00],
       [9.99999000e-01, 2.00000000e+00],
       [9.99999000e-01, 2.00000000e+00],
       [9.99883912e-01, 1.00000000e+00],
       [9.99999000e-01, 2.00000000e+00],
       [9.99999000e-01, 1.00000000e+00],
       [9.99656716e-01, 1.00000000e+00],
       [9.99999000e-01, 2.00000000e+00],
       [9.99999000e-01, 2.00000000e+00],
       [1.000000

# Why is initialization of weights important? <a class="anchor" id="5"></a>

Initialization of the weights or coefficients is crucial as it sets the starting values for the optimization algorithm used to estimate the model parameters. Proper initialization helps ensure the convergence of the optimization process and the stability of the estimated coefficients. 

**What happens if weights are not initialized properly?**

1. *Slow /Poor convergence:* <br>
   Poor initialization may lead to slow convergence or the algorithm getting stuck in local optima. 
2. *Degeneracy*: <br>
    If the model fails to converge, we may get unreliable results. This is called degeneracy. This can happen especially especially when the input features of the data are highly correlated.
4. *Stability issues:* <br>
   Well-initialized weights ensure that the optimization algorithm explores a reasonable parameter space, avoiding extreme or unrealistic estimates.If the weights are not initialized properly, model may explore the regions which have poor interpretability and may yield non-sensical results.
4. *Influence on regularization:* <br>
   If the weights are not initialized, then an $L1$ or $L2$ which depends on the weights can cause undesirable effects on the weights update.


# Code: Multiclass Logistic Regression

## Data Preparation

In [13]:
rootpath = "../../"

In [14]:
csvpath = os.path.join(rootpath, "assets/iris.csv")
df = pd.read_csv(csvpath)
num_samples, num_feats = df.shape
print(f"Total samples: {num_samples}")
print(f"Total features: {num_feats}")
feat_names = list(df)
print(f"Features: {feat_names}")

Total samples: 150
Total features: 5
Features: ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'species']


In [25]:
## Count the frequency
df.species.value_counts()

species
setosa        50
versicolor    50
virginica     50
Name: count, dtype: int64

In [30]:
# Create a unique string-->int label map


unique_labels = np.unique(df.species.to_numpy())
label_dict = {label: index for index, label in enumerate(unique_labels)}
